<a href="https://colab.research.google.com/github/tanakorn911/Intelligent-Agents/blob/main/67020982_Lab2_Tanakorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/aimacode/aima-python/refs/heads/master/agents.py

In [ ]:
!wget https://raw.githubusercontent.com/aimacode/aima-python/master/utils.py

In [ ]:
!pip install ipythonblocks

In [ ]:
from agents import *

import random

 Part 1: สร้างสุนัขและสวน (Simple Agent & Environment)

ในส่วนแรก เราจะสร้าง "สุนัขตาบอด" (BlindDog) ที่รับรู้ได้แค่ว่า "ตำแหน่งปัจจุบัน" มีอะไรอยู่ และทำได้แค่ "กิน" หรือ "ดื่ม"

1.1 สร้าง Class สุนัข (Agent)

พิมพ์โค้ดต่อไปนี้เพื่อกำหนดคุณสมบัติของสุนัข:

In [ ]:
class BlindDog(Agent):
    location = 1  # เริ่มต้นที่ตำแหน่ง 1 (โลก 1 มิติ)

    def movedown(self):
        self.location += 1

    def eat(self, thing):
        if isinstance(thing, Food):
            print(f"Dog: Ate food at {self.location}.")
            return True
        return False

    def drink(self, thing):
        if isinstance(thing, Water):
            print(f"Dog: Drank water at {self.location}.")
            return True
        return False

1.2 สร้าง Class อาหารและน้ำ (Things)

In [ ]:
class Food(Thing):
    pass

class Water(Thing):
    pass

1.3 สร้าง Class สวนสาธารณะ (Environment)

เราต้องกำหนดว่าเมื่อสุนัขทำ Action ต่างๆ สภาพแวดล้อมจะเปลี่ยนแปลงอย่างไร

In [ ]:
class Park(Environment):
    def percept(self, agent):
        '''ส่งรายการสิ่งของที่อยู่ในตำแหน่งเดียวกับ Agent'''
        things = self.list_things_at(agent.location)
        return things

    def execute_action(self, agent, action):
        '''เปลี่ยนแปลงสภาพแวดล้อมตามการกระทำของ Agent'''
        if action == "move down":
            print(f'{str(agent)[1:-1]} decided to {action} at location: {agent.location}')
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    self.delete_thing(items[0]) # กินแล้วต้องลบออกจากโลก
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    self.delete_thing(items[0]) # ดื่มแล้วลบออก

 Part 2: ใส่สมองให้สุนัข (The Agent Program)
ตัว Agent (ร่างกาย) พร้อมแล้ว แต่ยังขาด "สมอง" หรือโปรแกรมที่จะสั่งการ เราจะเขียนฟังก์ชัน program ที่รับค่า percepts (สิ่งที่มองเห็น/สัมผัส) แล้วส่งคืน action

2.1 เขียนโปรแกรมควบคุม

เงื่อนไข:

* ถ้าเจอ อาหาร -> ให้สั่ง 'eat'
* ถ้าเจอ น้ำ -> ให้สั่ง 'drink'
* ถ้าไม่เจออะไรเลย -> ให้สั่ง 'move down'

In [ ]:
def program(percepts):
    '''Returns an action based on the dog's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'

2.2 ทดสอบการจำลองสถานการณ์ (Simulation)

ลองนำทุกอย่างมารวมกัน แล้วดูว่าสุนัขของเราหาอาหารเจอหรือไม่

In [ ]:
park = Park()

# 2. สร้างสุนัข โดยใส่สมอง (program) เข้าไป
dog = BlindDog(program)

# 3. สร้างอาหารและน้ำ
dogfood = Food()
water = Water()

# 4. เอาของทุกอย่างใส่ลงในสวน (กำหนดพิกัด 1 มิติ)
park.add_thing(dog, 1)      # หมาอยู่ที่ 1
park.add_thing(dogfood, 5)  # อาหารอยู่ที่ 5
park.add_thing(water, 7)    # น้ำอยู่ที่ 7

# 5. รันระบบ 10 ก้าว
print("--- Simulation Start ---")
park.run(10)

🟠 Part 3: โลก 2 มิติ และสุนัขจอมพลัง (2D Environment)

ในโลกจริง สุนัขไม่ได้เดินเป็นเส้นตรงแค่ขึ้นกับลง เราจะเปลี่ยนสภาพแวดล้อมเป็น 2D (ตาราง Grid) และให้สุนัขเดินเลี้ยวซ้ายขวาได้

3.1 สร้างสุนัขจอมพลัง (EnergeticBlindDog)

สุนัขตัวนี้ฉลาดขึ้น คือรู้จักการ "เลี้ยว" (Turn) และการ "เดินหน้า" (Move Forward) แทนที่จะ Move down อย่างเดียว

In [ ]:
class EnergeticBlindDog(Agent):
    location = [0,1] # ตำแหน่งเป็น List [x, y]
    direction = Direction("down") # มีทิศทางที่หันหน้าอยู่

    def moveforward(self, success=True):
        if not success: return # ถ้าชนกำแพง จะไม่ขยับ

        # คำนวณพิกัดใหม่ตามทิศที่หัน
        if self.direction.direction == Direction.R: self.location[0] += 1
        elif self.direction.direction == Direction.L: self.location[0] -= 1
        elif self.direction.direction == Direction.D: self.location[1] += 1
        elif self.direction.direction == Direction.U: self.location[1] -= 1

    def turn(self, d):
        self.direction = self.direction + d # เปลี่ยนทิศ

    def eat(self, thing):
        if isinstance(thing, Food): return True
        return False

    def drink(self, thing):
        if isinstance(thing, Water): return True
        return False

3.2 สร้างสมองที่ซับซ้อนขึ้น (Random Movement Logic)

เนื่องจากสุนัขตาบอด มันไม่รู้ว่าอาหารอยู่ไหน มันจึงต้อง "เดินสุ่ม" แต่ถ้ามันเดินชนขอบรั้วสวน (Bump) มันต้องรู้จักเลี้ยวกลับ

In [ ]:
def energetic_program(percepts):
    for p in percepts:
        if isinstance(p, Food): return 'eat'
        elif isinstance(p, Water): return 'drink'

        # ถ้าเจอขอบรั้ว (Bump) ให้สุ่มเลี้ยวซ้ายหรือขวา
        if isinstance(p, Bump):
            choice = random.choice((1, 2)) # 1=ขวา, 2=ซ้าย
            if choice == 1: return 'turnright'
            else: return 'turnleft'

    # ถ้าทางโล่ง: สุ่มว่าจะ เลี้ยว หรือ เดินหน้า
    # 1=ขวา, 2=ซ้าย, 3-4=เดินหน้า (โอกาสเดินหน้า 50%)
    choice = random.choice((1, 2, 3, 4))
    if choice == 1: return 'turnright'
    elif choice == 2: return 'turnleft'
    else: return 'moveforward'

3.3 สร้างสวน 2 มิติ (Park2D)

เราจะใช้ GraphicEnvironment ซึ่งเป็น Class พิเศษใน agents.py ที่รองรับระบบพิกัดและการชนขอบ (Boundaries)

โค้ดส่วนนี้มีความสำคัญมากเพราะเราต้องเปลี่ยนจาก Environment ธรรมดา มาเป็น GraphicEnvironment (ที่มีระบบพิกัด x, y) และต้องเขียน Logic เพื่อจัดการเรื่อง "กำแพง" (Boundaries) ด้วยครับ

In [ ]:
import copy # จำเป็นต้องใช้เพื่อ copy ตำแหน่งมาคำนวณล่วงหน้า

class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''
        ฟังก์ชันรับรู้สภาพแวดล้อม (Overridden)
        หน้าที่พิเศษ: ตรวจสอบว่าข้างหน้ามีกำแพงไหม? ถ้ามีให้ส่ง Bump() ไปบอก Agent
        '''
        # 1. ดูของที่อยู่ที่เท้า (ตำแหน่งปัจจุบัน)
        things = self.list_things_at(agent.location)

        # 2. ตรวจสอบกำแพง (Look ahead)
        # จำลองตำแหน่งถัดไป โดยดูทิศที่หมาหันหน้าอยู่
        loc = copy.deepcopy(agent.location)
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1

        # 3. ถ้าตำแหน่งถัดไป (loc) อยู่นอกขอบเขตสวน -> ให้เพิ่ม "Bump" (ชน) เข้าไปในการรับรู้
        if not self.is_inbounds(loc):
            things.append(Bump())

        return things

    def execute_action(self, agent, action):
        '''
        ฟังก์ชันตอบสนองการกระทำ (Overridden)
        เพิ่มการรองรับการเลี้ยว (Turn) และเดินหน้า (Move Forward)
        '''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R) # สั่ง Agent หันขวา

        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L) # สั่ง Agent หันซ้าย

        elif action == 'moveforward':
            # Agent จะเดินหน้าไปเอง โดยเช็คทิศทางจากตัวแปร direction ในตัวมัน
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()

        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'.format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) # กินหมดแล้วลบออกจากโลก

        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'.format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) # ดื่มหมดแล้วลบออกจากโลก

    def is_done(self):
        '''
        จบการทำงานเมื่อไหร่?
        1. อาหารหรือน้ำหมดโลก (no_edibles)
        2. หรือ สัตว์เลี้ยงตายหมด (dead_agents)
        '''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

3.4 ทดสอบระบบ 2D

In [ ]:
# สร้างสวนขนาดกว้าง 5 สูง 5
# กำหนดสี: หมา=แดง, น้ำ=ฟ้า, อาหาร=ส้ม
park_2d = Park2D(5, 5, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})

dog_2d = EnergeticBlindDog(energetic_program)
dogfood = Food()
water = Water()

# เพิ่มของลงในพิกัดต่างๆ [x, y]
park_2d.add_thing(dog_2d, [0,0])
park_2d.add_thing(dogfood, [1,2])
park_2d.add_thing(water, [0,1])
park_2d.add_thing(water, [2,2])

print("Starting 2D Simulation...")
park_2d.run(20)

สมองหมา v2

In [ ]:
import random # อย่าลืม import นะครับ

def energetic_program_v2(percepts):
    for p in percepts:
        # 1.Priority แรก: เจอของกิน/ดื่ม ให้จัดการก่อน
        if isinstance(p, Food): return 'eat'
        elif isinstance(p, Water): return 'drink'

        # 2. Priority สอง: ถ้าชนกำแพง (Bump) ต้องเลี้ยวหนี (อันนี้คงเดิมไว้ 50/50)
        if isinstance(p, Bump):
            choice = random.choice((1, 2))
            if choice == 1: return 'turnright'
            else: return 'turnleft'

    # --- 3. Priority สุดท้าย: เดินสำรวจ (ส่วนที่แก้ไข) ---

    # สุ่มค่า 0.0 ถึง 1.0
    r = random.random()

    if r < 0.1:       # 10% (0.0 ถึง 0.1)
        return 'turnright'

    elif r < 0.2:     # 10% (0.1 ถึง 0.2)
        return 'turnleft'

    else:             # 80% (ที่เหลือทั้งหมด: 0.2 ถึง 1.0)
        return 'moveforward'

📝 แบบฝึกหัดท้ายการทดลอง (Assignment)

1. Modify Probability: ในฟังก์ชัน energetic_program ปัจจุบันสุนัขมีโอกาสเดินหน้า 50% เลี้ยวซ้าย 25% เลี้ยวขวา 25%
  * จงแก้ Code ให้สุนัขมีนิสัย "ชอบเดินหน้ามากกว่าเลี้ยว" โดยให้โอกาสเดินหน้าเป็น 80% และเลี้ยวซ้าย/ขวา อย่างละ 10%
  * (แปะ Code ที่แก้แล้วลงในรายงาน)
2. Add Obstacle:
  * จงสร้าง Class ใหม่ชื่อ Rock (หิน) ที่สืบทอดมาจาก Thing
  * แก้ไข program ของสุนัข: ถ้าสุนัขเดินไปเจอ Rock (Percept เห็นหิน) ให้สุนัขทำ Action อะไรก็ได้ที่ไม่ใช่การเดินชน (เช่น ให้เลี้ยวกลับ หรือเดินหนี)
3. Analysis:
  * คิดว่า EnergeticBlindDog (ที่เดินสุ่ม) เป็นวิธีที่มีประสิทธิภาพที่สุดในการหาอาหารหรือไม่? ถ้าเราอยากให้มันหาอาหารได้เร็วขึ้นโดยที่ยังตาบอดอยู่ เราควรปรับปรุง Logic การเดินอย่างไร? (อธิบายแนวคิด)

1) Modify Probability

In [ ]:
def energetic_program_v2(percepts):
    for p in percepts:
        # 1. เจออาหารหรือน้ำ -> กิน/ดื่มก่อน
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'

        # 2. ถ้าชนกำแพง -> เลี้ยวหนีแบบสุ่ม
        if isinstance(p, Bump):
            choice = random.choice((1, 2))  # 1=ขวา, 2=ซ้าย
            if choice == 1:
                return 'turnright'
            else:
                return 'turnleft'

    r = random.random()  # ได้ค่า 0.0 - 1.0

    if r < 0.1:          # 10%
        return 'turnright'
    elif r < 0.2:        # 10%
        return 'turnleft'
    else:                # 80%
        return 'moveforward'


2) Add Obstacle

จงสร้าง Class ใหม่ชื่อ Rock (หิน) ที่สืบทอดมาจาก Thing

In [ ]:
class Rock(Thing):
    pass

แก้ไข program ของสุนัข: ถ้าสุนัขเดินไปเจอ Rock (Percept เห็นหิน) ให้สุนัขทำ Action อะไรก็ได้ที่ไม่ใช่การเดินชน (เช่น ให้เลี้ยวกลับ หรือเดินหนี)

In [ ]:
def energetic_program(percepts):
    for p in percepts:

        # ถ้าเป็นอาหาร → กิน
        if isinstance(p, Food):
            return 'eat'

        # ถ้าเป็นน้ำ → ดื่ม
        elif isinstance(p, Water):
            return 'drink'

        # ถ้าเป็นกำแพง (Bump) → เลี้ยวสุ่ม
        elif isinstance(p, Bump):
            return random.choice(['turnleft', 'turnright'])

        # ถ้าเจอหิน → ห้ามเดินชน → เลี้ยวกลับหรือเลี่ยง
        elif isinstance(p, Rock):
            return random.choice(['turnleft', 'turnright', 'turnleft'])
            # (เพิ่ม turnleft ซ้ำเพื่อ bias ให้เลี่ยวมากกว่าเดินชน)

    # ถ้าทางโล่ง → โอกาสสุ่มเลี้ยวหรือเดินหน้า
    choice = random.choice((1, 2, 3, 4))
    if choice == 1: return 'turnright'
    elif choice == 2: return 'turnleft'
    else: return 'moveforward'


In [ ]:
class Rock(Thing):
    pass


park_2d = Park2D(5, 5, color={
    'EnergeticBlindDog': (200,0,0),
    'Water': (0, 200, 200),
    'Food': (230, 115, 40),
    'Rock': (120,120,120)
})

dog_2d = EnergeticBlindDog(energetic_program)
dogfood = Food()
water = Water()

rock1 = Rock()
rock2 = Rock()

park_2d.add_thing(dog_2d, [0,0])
park_2d.add_thing(dogfood, [1,2])
park_2d.add_thing(water, [0,1])
park_2d.add_thing(water, [2,2])

park_2d.add_thing(rock1, [1,1])
park_2d.add_thing(rock2, [3,2])

print("Starting 2D Simulation...")
park_2d.run(20)


3. Analysis:
คิดว่า EnergeticBlindDog (ที่เดินสุ่ม) เป็นวิธีที่มีประสิทธิภาพที่สุดในการหาอาหารหรือไม่? ถ้าเราอยากให้มันหาอาหารได้เร็วขึ้นโดยที่ยังตาบอดอยู่ เราควรปรับปรุง Logic การเดินอย่างไร? (อธิบายแนวคิด)

1. ใช้การเดินแบบ systematic walk
เช่น เดินกวาดพื้นที่เป็น pattern /
รูปตัว S /
รูปงู /
รูปกากบาท
หรือ zig-zag scanning
วิธีนี้ทำให้มั่นใจว่า "พื้นที่ทุกจุดจะถูกสำรวจ" อย่างเป็นระบบ ไม่ซ้ำซ้อน

2. ใช้ Memory ช่วยจำตำแหน่งที่เคยไปแล้ว
แม้หมาตาบอด แต่มันสามารถจำตำแหน่งที่เคยผ่าน → ไม่ต้องกลับไปซ้ำ
เช่น เก็บตำแหน่ง visited ใน list/set

3. ใช้การเลี้ยวแบบหลีกเลี่ยงการวนลูป
เช่น ถ้าเลี้ยวซ้ายแล้วเกิดวนซ้ำ → เปลี่ยนเป็นเลี้ยวขวาอัตโนมัติ

4. ใช้ Heuristic แม้ไม่เห็นอาหาร
เช่น ถ้าเดินชนกำแพงบ่อย → ปรับทิศ ถ้าอยู่มุม → เดินออกจากมุม